This is the scrap the metadata of the story on webnovel.com

In [ ]:
import os
import re
import json
import time
import requests
import numpy as np
import pandas as pd
from lxml import html
from tqdm import tqdm
from bs4 import BeautifulSoup as bs


In [ ]:
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }

In [ ]:
bookList = pd.read_csv('data/bookList.csv')
bookList = bookList.dropna(subset=['qidianUrl'])

In [ ]:
bookList.head()

,qidianUrl,webnovelUrl,qidianBookId,webnovelBookId
0,https://www.qidian.com/book/1003692682/,https://www.webnovel.com/book/super-gene_80224...,1.003693e+09,8022472105002805
1,https://www.qidian.com/book/1010868264/,https://www.webnovel.com/book/lord-of-mysterie...,1.010868e+09,11022733006234505
2,https://www.qidian.com/book/1009480992/,https://www.webnovel.com/book/the-legendary-me...,1.009481e+09,8662546605001405
3,https://www.qidian.com/book/3471716/,https://www.webnovel.com/book/reincarnation-of...,3.471716e+06,8527113906000305
4,https://www.qidian.com/book/1004179514/,https://www.webnovel.com/book/library-of-heave...,1.004180e+09,6831850602000905


# Requests Solution

It only collects what we see on the page.

In [ ]:
#scrape hompeage to collect book information

hpage = 'https://www.webnovel.com/stories/novel'

response = requests.get(hpage, headers=headers)
soup = bs(response.text,'html')
books = soup.find_all('script',{"type":"application/ld+json"})
script = json.loads(books[1].text)
book_list = script.get("itemListElement")
book_df = pd.DataFrame(book_list)


In [ ]:
book_df.head()

,@type,position,url,name,image,author,publisher
0,ListItem,1,https://www.webnovel.com/book/supreme-magus_12...,Supreme Magus,http://book-pic.webnovel.com/bookcover/1282087...,"{'@type': 'Person', 'name': 'Legion20'}","{'@type': 'Organization', 'name': 'WebNovel'}"
1,ListItem,2,https://www.webnovel.com/book/my-vampire-syste...,My Vampire System,http://book-pic.webnovel.com/bookcover/1670936...,"{'@type': 'Person', 'name': 'JKSManga'}","{'@type': 'Organization', 'name': 'WebNovel'}"
2,ListItem,3,https://www.webnovel.com/book/blood-warlock-su...,Blood Warlock: Succubus Partner in the Apocalypse,http://book-pic.webnovel.com/bookcover/2013475...,"{'@type': 'Person', 'name': 'XIETIAN'}","{'@type': 'Organization', 'name': 'WebNovel'}"
3,ListItem,4,https://www.webnovel.com/book/shadow-slave_221...,Shadow Slave,http://book-pic.webnovel.com/bookcover/2219654...,"{'@type': 'Person', 'name': 'Guiltythree'}","{'@type': 'Organization', 'name': 'WebNovel'}"
4,ListItem,5,https://www.webnovel.com/book/under-the-oak-tr...,Under the Oak Tree,http://book-pic.webnovel.com/bookcover/2194958...,"{'@type': 'Person', 'name': 'Suji Kim'}","{'@type': 'Organization', 'name': 'WebNovel'}"


The first try to see what kind of metadata can i get from scrapping

In [ ]:
#auxiliary functions

def scrape_book_metadata(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        try:
          soup = bs(response.text,'html')
          script = soup.find_all('script',{"type":"application/ld+json"})
          data = json.loads(books[1].text)
          return data
        except json.decoder.JSONDecodeError:
           print("Failed to decode JSON. Response content is not valid JSON.")
           print("Returning Soup instead:")
           soup = bs(response.text,'html')
           #print(soup)
           return soup
    else:
        print("Failed to fetch book metadata.")
        return None

In [ ]:

# Example usage:
book_url = book_df['url'].iloc[0]
print(book_url)
# Fetch book metadata
response = requests.get(book_urresponse = requests.get(book_urscript[1].textl, headers=headers)
script[1].textl, headers=headers)
soup = bs(response.text,'html')
script = soup.find_all('script',{"type":"application/ld+json"})


In [ ]:
book_meta = json.loads(script[1].text.replace('\n',''))[0]


In [ ]:
book_meta.keys()

dict_keys(['@context', '@type', 'mainEntityOfPage', 'headline', 'name', 'genre', 'image', 'bookFormat', 'datePublished', 'dateModified', 'author', 'copyrightHolder', 'publisher', 'description', 'aggregateRating', 'review', 'potentialAction'])

In [ ]:
pd.DataFrame(book_meta['review'])

,@type,author,datePublished,reviewBody,reviewRating
0,Review,"{'@type': 'Person', 'name': 'CPT_Ling_Ling'}",2022-06-16T17:25:49.000Z,I&#39;ve been reading this book for a while no...,"{'@type': 'Rating', 'ratingValue': '5'}"
1,Review,"{'@type': 'Person', 'name': 'ThoMiCroN'}",2022-08-09T15:42:39.000Z,By far the best book on this platform. Remembe...,"{'@type': 'Rating', 'ratingValue': '5'}"
2,Review,"{'@type': 'Person', 'name': 'WrongerMonk_10'}",2022-08-16T16:35:21.000Z,"Supreme Magus is, hands down, one of the best ...","{'@type': 'Rating', 'ratingValue': '5'}"
3,Review,"{'@type': 'Person', 'name': 'Legion20'}",2019-01-23T14:30:29.000Z,"Shameless 5 star review from the author, lol. ...","{'@type': 'Rating', 'ratingValue': '5'}"
4,Review,"{'@type': 'Person', 'name': 'KylarDrake'}",2021-04-18T03:39:38.000Z,I can’t recommend this book enough. One of the...,"{'@type': 'Rating', 'ratingValue': '5'}"


In [ ]:
book_meta['aggregateRating']

{'@type': 'AggregateRating',
 'bestRating': '5.0',
 'ratingValue': '4.81',
 'ratingCount': '6105'}

# Collecting Meta

In [ ]:
def get_meta(book_url):
  response = requests.get(book_url,headers=headers)
  encoding = response.headers['Content-Type'].split('; ')[1].split('=')[-1]

  soup = bs(response.text,'html')

  body_text = bytes.decode(response.content, encoding)
  tree = html.fromstring(body_text)

  meta = soup.find_all('script', type='application/ld+json')[1]
  meta_dict = json.loads(meta.text.replace('\n','').replace('\r',''))[0]

  authorProfile = None
  if tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/address/h2/a"):
    authorProfile = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/address/h2/a")[0].get('href')

  if 'Chapters' in tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[1]/span")[0].text:
    numberOfChapters = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[1]/span")[0].text
  else:
    numberOfChapters = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[2]/span")[0].text

  if 'Views' in tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[2]/span")[0].text:
    numberOfViews = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[2]/span")[0].text
  else:
    numberOfViews = tree.xpath("/html/body/div[1]/div[3]/div/div/div[2]/h2/strong[3]/span")[0].text

  synopsis = soup.find_all('div',{'class':"j_synopsis"})[0].text

  return [meta_dict['name'],meta_dict['genre'],meta_dict['datePublished'],meta_dict['dateModified'],meta_dict['copyrightHolder'],meta_dict['description'],
                   meta_dict['aggregateRating']['bestRating'],meta_dict['aggregateRating']['ratingValue'],meta_dict['aggregateRating']['ratingCount'],meta_dict['author']['name'],
                   authorProfile,numberOfChapters,numberOfViews,synopsis]



In [ ]:
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }
meta_column_names = ['webnovelId','webnovelName', 'webnovelGenre', 'webnovelDatePublished', 'webnovelDateModified', 'webnovelCopyrightHolder', 'webnovelDescription','webnovelBestRating',
                     'webnovelRatingValue','webnovelRatingCount','webnovelAuthor','webnovelAuthorProfile','webnovelNumberOfChapters','webnovelNumberOfViews','webnovelSynopsis']
meta_info = []
for i,row in tqdm(bookList.iterrows()):
  url = row['webnovelUrl']
  id = row['webnovelBookId']
  meta_info.append([id]+get_meta(url))



meta_df = pd.DataFrame(meta_info,columns=meta_column_names)
meta_df.to_csv('webnovelMeta.csv',index=False)

# Collecting Reviews and Replies

In [ ]:
#for comments
#pageSize can be max 100.

def get_webnovelReviews(bookId,token):
  reviews_df = pd.DataFrame()
  url = "https://www.webnovel.com/go/pcm/bookReview/get-reviews"
  payload = ""
  pageIndex = 1
  headers = {
      "cookie": "_csrfToken=8850f678-3bca-4230-88aa-7ad5443f4079; webnovel_uuid=1713274934_6925865",
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
      "Cookie": "_csrfToken=" + token
  }

  while True:
    querystring = {"":["",""],"_csrfToken":token,"bookId":bookId,
                  "pageIndex":str(pageIndex),"pageSize":"100","orderBy":"1","novelType":"0","needSummary":"1","_":"1713277524976"}
    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    json_response = json.loads(response.text)
    json_reviews = json_response['data']['bookReviewInfos']
    json_df = pd.DataFrame(json_reviews)
    if len(json_df) == 0:
      break
    reviews_df = pd.concat([reviews_df,json_df],ignore_index=True)
    pageIndex += 1

  #get the book info only once and add it to the meta data
  json_book_info = json_response['data']['bookStatisticsInfo']

  return reviews_df,json_book_info


In [ ]:
token = "36c04316-370e-45ec-87a1-001cd336f6fe"
additional_book_meta = []
for bookId in tqdm(bookList['webnovelBookId']):
  review_file_name = 'data/' + str(bookId) + '.csv'
  info,reviews = get_webnovelReviews(bookId,token)
  reviews.to_csv(review_file_name,index=False)
  info.update({'webnovelBookId':bookId})
  additional_book_meta.append(info)

df = pd.DataFrame(additional_book_meta)
df.to_csv('data/webnovelMetaAdditional.csv',index=False)

100%|██████████| 120/120 [01:48<00:00,  1.11it/s]


In [ ]:
#join additional Meta with the meta

meta = pd.read_csv('data/webnovelMeta.csv')
additional_meta = pd.read_csv('data/webnovelMetaAdditional.csv')
metaTotal = pd.merge(left=meta,right=additional_meta,left_on='webnovelId',right_on='webnovelBookId')

In [ ]:
#cleaning
nc = metaTotal['webnovelNumberOfChapters'].apply(lambda x:x.split(' ')[0])
metaTotal['webnovelNumberOfChapters'] = nc.str.replace(',','')

nv = metaTotal['webnovelNumberOfViews'].apply(lambda x: x.split(' ')[0])
metaTotal['webnovelNumberOfViews'] = nv


In [ ]:
metaTotal = metaTotal.drop(['webnovelRatingValue','webnovelRatingCount','bookName','webnovelBookId'],axis=1)

In [ ]:
metaTotal.to_csv('data/webnovelMetaTotal.csv',index=False)

In [ ]:
def get_webnovelReplies(bookId,reviewId, token):
  replies_df = pd.DataFrame()
  url = "https://www.webnovel.com/go/pcm/bookReview/detail"
  payload = ""
  headers = {
    "cookie": "_csrfToken=8850f678-3bca-4230-88aa-7ad5443f4079; webnovel_uuid=1713274934_6925865",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Cookie": "_csrfToken=" + token
  }
  pageIndex = 1

  while True:
    querystring = {"":["",""],"_csrfToken":token,"reviewId":reviewId,
                  "pageIndex":str(pageIndex), "bookId":bookId,"_":"1713291897856"}

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

    json_response = json.loads(response.text)
    json_replies = json_response['data']['replyItems']
    json_df = pd.DataFrame(json_replies)
    if len(json_df) == 0:
      break
    replies_df = pd.concat([replies_df,json_df],ignore_index=True)
    pageIndex += 1
    replies_df['reviewId'] = reviewId
  return replies_df

In [ ]:
token = "36c04316-370e-45ec-87a1-001cd336f6fe"

for bookId in tqdm(bookList['webnovelBookId'][80:120]):
  reviews = pd.read_csv('data/reviews/'+str(bookId)+'.csv')
  replies = pd.DataFrame()
  for i,row in reviews.iterrows():
    reviewId = row['reviewId']
    if row['replyAmount']:
      replies = pd.concat([replies,get_webnovelReplies(bookId,reviewId,token)])

  replies.to_csv('data/replies/'+str(bookId)+'.csv',index=False)

100%|██████████| 40/40 [26:34<00:00, 39.85s/it]


# Collecting User Profiles

In [ ]:
bookList['webnovelUrl'].iloc[0]

'https://www.webnovel.com/book/super-gene_8022472105002805'

In [ ]:
# collect all user ids
# it is very fast: 1.5 minutes
userIds = list()
for bookId in bookList['webnovelBookId']:
  review_path = os.path.join('data/webnovelReviews/',str(bookId)+'.csv')
  reply_path = os.path.join('data/webnovelReplies/',str(bookId)+'.csv')
  reviews = pd.read_csv(review_path)
  userIds += list(reviews['userId'])
  replies = pd.read_csv(reply_path)
  userIds += list(replies['userId'])



In [ ]:
with open('userIds.txt','w') as fout:
  for id in userIds:
    fout.writelines(str(id)+'\n')

In [ ]:
def get_userInfo(userId):
  user_url = 'https://www.webnovel.com/profile/' + str(userId)
  response = requests.get(user_url,headers=headers)
  soup = bs(response.text,'html')
  user_stats = soup.find_all('ul',{'class':'stat'})[0]
  user_stats = user_stats.text.replace(' ','')
  user_details = soup.find_all('section',{'class':'pb32'})[0]
  wd = re.search(r"([0-9]+)d",user_stats)
  rh = re.search(r"([0-9,\.]+)h",user_stats)
  rb = re.search(r"([0-9]+)R",user_stats)
  name = user_details.find_all('h3')[0].text

  if wd:
    writingDays = wd.group(1)
  else:
    writingDays = None
  if rh:
    readingHours = rh.group(1)
  else:
    readingHours = None
  if rb:
    readBooks = rb.group(1)
  else:
    readBooks = None

  if user_details.find_all('i'):
    gender = user_details.find_all('i')[0].text
  else:
    gender = None

  level = user_details.find_all('strong',{'class':"g_lv"})[0].text.split('LV ')[1]

  if user_details.find_all('p'):
    blurb = user_details.find_all('p')[0].text
  else:
    blurb = None

  if user_details.find_all('address'):
    text = user_details.find_all('address')[0].text
    if 'Joined' in text:
      dateJoined = text.split('Joined')[0].strip()
      location = text.split('Joined')[1].strip()
    else:
      dateJoined = '1970-01-01'
      location = text.strip()
  else:
    dateJoined = '1970-01-01'
    location = 'Unknown'


  user_info = {'userId':userId,'Name':name,'Gender':gender,'Level':level,'writingDays':writingDays,'readingHours':readingHours,
               'readBooks':readBooks,'Description':blurb,'dateJoined':dateJoined,'Location':location}

  return user_info

In [ ]:
all_user_info = []
uniqueUserIds = list(set(userIds))
for uid in tqdm(uniqueUserIds):
  all_user_info.append(get_userInfo(uid))
user_df = pd.DataFrame(all_user_info)
user_df.to_csv('data/webnovelUserProfiles.csv',index=False)

NameError: name 'userIds' is not defined

The user profiles were collected by the server. i combined them intoma single file here.

In [ ]:
profile1 = pd.read_csv("/content/drive/MyDrive/CITCalls/Ze/data/webnovelUserProfiles.csv")
profile2 = pd.read_csv("/content/drive/MyDrive/CITCalls/Ze/data/webnovelUserProfiles2.csv")
profile3 = pd.read_csv("/content/drive/MyDrive/CITCalls/Ze/data/webnovelUserProfiles3.csv")
profile4 = pd.read_csv("/content/drive/MyDrive/CITCalls/Ze/data/webnovelUserProfiles4.csv")

In [ ]:
profile = pd.concat([profile1,profile2,profile3,profile4],ignore_index=True)
profile.shape

(128662, 10)

In [ ]:
profile.to_csv('/content/drive/MyDrive/CITCalls/Ze/data/webnovelUserProfiles.csv',index=False)

# Selenium

The information from the next two cells are retrieved from https://nariyoo.com/python-how-to-run-selenium-in-google-colab/. Ze provided a list of books to work on. The list collected by this method is not needed, but Ze thinks it can be used later.

NOT USED!!!


In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium
pip install selenium-wire
pip install chromedriver-autoinstaller

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By

from seleniumwire import webdriver as webdriversw
from seleniumwire.utils import decode as decodesw

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# setup chrome options for sw
chrome_options = webdriversw.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

driver = webdriversw.Chrome(options=chrome_options)

## List of Books

In [ ]:
def get_html_content(url,driver):
  driver.get(url)
  driver.refresh()
  driver.refresh()

  SCROLL_PAUSE_TIME = 2

  # Get scroll height
  last_height = driver.execute_script("return document.body.scrollHeight")

  book_list = []

  # does ~50 scroll downs
  while True:
      # Scroll down to bottom
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

      # Wait to load page
      time.sleep(SCROLL_PAUSE_TIME)

      # Calculate new scroll height and compare with last scroll height
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height
  return driver.page_source


def get_book_list(content):
  soup = bs(content,'html')
  book_list = soup.find_all('li',{"class":"fl w50% bsbb pl20 pr20 pt8 pb8"})
  book_anchors = [book.find_all('a',{"class":"g_thumb _auto"})[0] for book in book_list]
  return book_anchors

def get_book_info(book):
  return [book['href'],book['title']]



### Collection Process
1. Generate the urls per book type
2. Collect data using selenium

In [ ]:
#novel_urls
novel_urls = ["https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=1", #popular
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=2", #recommended
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=3", #most_collections
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=4", #rating
            "https://www.webnovel.com/stories/novel?sourceType=0&bookStatus=0&orderBy=5"]
novel_genres = ['eastern-male','fantasy-male','horror-male','action-male','realistic-male','acg-male',
               'urban-male','games-male','scifi-male','sports-male','war-male','history-male','fantasy-female',
               'teen-female','scifi-female','urban-female','history-female','lgbt-female','general-female']
for g in novel_genres:
  for i in range(1,6):
    novel_urls.append(f"https://www.webnovel.com/stories/novel-{g}?bookStatus=0&orderBy={i}")


In [ ]:
#comic urls
comic_urls = ["https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=1", #popular
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=2", #recommended
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=3", #most_collections
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=4", #rating
            "https://www.webnovel.com/stories/comic?sourceType=0&bookStatus=0&orderBy=5"]
comic_genres = ['action','fantasy','harem','magic','eastern','scifi','sports','inspiring','comedy','drama',
'mystery','romance','lgbt','urban','transmigration','school','adventure','history','horror',
'wuxia','slice-of-life','cooking','diabolical']

for g in comic_genres:
  for i in range(1,6):
    comic_urls.append(f"https://www.webnovel.com/stories/comic-{g}?bookStatus=0&orderBy={i}")

In [ ]:
#fanfic_urls

fanfic_urls = ["https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=1", #popular
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=2", #recommended
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=3", #most_collections
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=4", #rating
            "https://www.webnovel.com/stories/fanfic?sourceType=0&bookStatus=0&orderBy=5"]
fanfic_genres = ['video-games','music-bands','book-literature','theater','anime-comics','celebrities','movies','tv','others']
for g in fanfic_genres:
  for i in range(1,6):
    fanfic_urls.append(f"https://www.webnovel.com/stories/fanfic-{g}?bookStatus=0&orderBy={i}")

In [ ]:
# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)
#url_list = novel_urls + comic_urls + fanfic_urls
book_info = []

for url in tqdm(fanfic_urls):
  html_content = get_html_content(url,driver)
  books = get_book_list(html_content)
  for book in books:
    book_info.append(get_book_info(book))

driver.close()

#save the information
#book_df = pd.DataFrame(book_info,columns=['Url','Title'])
#book_df.to_csv('webnovel_comic_list_all.csv',index=False)
#print(book_df.shape)
#book_df2 = book_df.drop_duplicates('Url')
#print(book_df2.shape)
#book_df2.to_csv('webnovel_comic_list.csv',index=False)

### Analyse the collected data

In [ ]:
#connect the to the drive and change to the folder
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#change the folder
cd /content/drive/MyDrive/CITCalls/Ze

/content/drive/MyDrive/CITCalls/Ze


In [ ]:
#with duplicates
novels_all = pd.read_csv('data/webnovel_novel_list_all.csv')
comic_all = pd.read_csv('data/webnovel_comic_list_all.csv')
fanfic_all = pd.read_csv('data/webnovel_fanfic_list_all.csv')
#without duplicates
novels = pd.read_csv('data/webnovel_novel_list.csv')
comic = pd.read_csv('data/webnovel_comic_list.csv')
fanfic = pd.read_csv('data/webnovel_fanfic_list.csv')


In [ ]:
novels_all = novels_all.assign(Type='Novel')
novels = novels.assign(Type='Novel')
comic_all = comic_all.assign(Type='Comic')
comic = comic.assign(Type='Comic')
fanfic_all = fanfic_all.assign(Type='Fan-Fic')
fanfic = fanfic.assign(Type='Fan-Fic')

In [ ]:
webnovels_df = pd.concat([novels,comic,fanfic],ignore_index=True)
webnovels_df.drop_duplicates('Url')
webnovels_df.to_csv('webnovels.csv',index=False)
print(webnovels_df.shape)

(58997, 3)


In [ ]:
webnovels_df.head()

,Url,Title,Type
0,/book/supreme-magus_12820870105509205,Supreme Magus,Novel
1,/book/my-vampire-system_16709365405930105,My Vampire System,Novel
2,/book/blood-warlock-succubus-partner-in-the-ap...,Blood Warlock: Succubus Partner in the Apocalypse,Novel
3,/book/shadow-slave_22196546206090805,Shadow Slave,Novel
4,/book/under-the-oak-tree_21949582605391005,Under the Oak Tree,Novel


## Anotomy of a Single Book

In [ ]:
#thi is to extract tokens
book_list_df = book_list_df.replace(np.nan,'')
qidianPattern = 'book\/([0-9]*)\/'
webnovelPattern = '_([0-9]*)'
book_list_df['qidianBookId'] = book_list_df['qidianUrl'].apply(lambda x: re.findall(qidianPattern,x)[0] if re.findall(qidianPattern,x) else None)
book_list_df['webnovelBookId'] = book_list_df['webnovelUrl'].apply(lambda x: re.findall(webnovelPattern,x)[0] if re.findall(webnovelPattern,x) else None)